# Plastic Optimization

We attempt to design new plastic molecule using LDM.
Our main goal is on
1. Straw
2. ?

## Setup enviroment
!git clone https://github.com/Ahnd6474/KSEF

%cd your_shit/GitHub/KSEF

!pip install -r requirements.txt

In [7]:
import pandas as pd
from __future__ import annotations

import os
import pickle
from pathlib import Path
from typing import Iterable, Iterator, Optional
import pyarrow
import torch
from geoldm.configs import get_dataset_info
from geoldm.qm9 import dataset, load_model, sampling, visualize_molecule_3d

## Read Data

In [8]:
df=pd.read_parquet('data/plastic.parquet')
df

,comp,Tg,Tm,Td,YM,TS_y,TS_b,eps_b,perm_O2,perm_CO2,perm_He,perm_N2,perm_CH4,perm_H2,smiles,num_side,num_back,end_group,names
0,0.0,246.341049,354.051544,524.699646,773.208252,25.838213,32.409279,103.836082,-0.074397,-0.109570,0.493981,-0.071000,-0.061771,0.296051,"[[*]OCCC(=O)[*], [*]OCCC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, ]","[pha, pha]"
1,0.1,313.798401,449.257751,502.616241,1866.776855,44.379429,38.010651,7.215131,-0.043089,-0.006026,0.462577,-0.049233,-0.031425,0.342873,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
2,0.2,306.681305,446.618713,502.605896,1769.267456,44.743877,37.998493,8.727348,-0.050900,-0.028856,0.455602,-0.060321,-0.039934,0.322175,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
3,0.3,298.431488,442.813019,503.664642,1653.973633,44.287182,37.676380,10.997544,-0.057812,-0.048113,0.458959,-0.069126,-0.047438,0.314363,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
4,0.4,290.073944,436.398560,505.013275,1508.705688,42.840149,36.907192,14.239414,-0.065853,-0.065725,0.461121,-0.075789,-0.055182,0.302125,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373498,0.0,325.252380,462.438263,502.465759,2680.520020,39.839355,68.497833,30.515642,0.011635,-0.060768,0.179605,0.002471,0.001415,0.056135,"[[*]CC([*])O, [*]CC([*])O]","[nan, nan]","[nan, nan]","[None, None]","[PVA, PVA]"
1373499,0.0,369.788818,509.966248,587.491455,2227.901367,33.554047,37.865055,1.950041,2.547638,11.337365,21.572718,0.559539,0.881040,24.865118,"[[*]CC([*])c1ccccc1, [*]CC([*])c1ccccc1]","[nan, nan]","[nan, nan]","[None, None]","[PS, PS]"
1373500,0.0,344.375183,457.250885,574.055664,1724.992798,50.262169,46.697941,27.839941,0.099392,0.332474,1.581858,-0.005775,0.070204,1.482780,"[[*]CC([*])Cl, [*]CC([*])Cl]","[nan, nan]","[nan, nan]","[None, None]","[PVC, PVC]"
1373501,0.0,393.653473,536.671326,700.403137,1942.565186,44.805931,65.647507,27.729816,0.016741,0.138303,1.767082,-0.054595,-0.069961,1.336078,"[[*]CCOC(=O)c1ccc2cc(C(=O)O[*])ccc2c1, [*]CCOC...","[nan, nan]","[nan, nan]","[None, None]","[PEN, PEN]"


In [10]:
df.describe()

,comp,Tg,Tm,Td,YM,TS_y,TS_b,eps_b,perm_O2,perm_CO2,perm_He,perm_N2,perm_CH4,perm_H2
count,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06
mean,4.997987e-01,2.725839e+02,3.529060e+02,5.769558e+02,5.405507e+02,2.073843e+01,2.460826e+01,2.115185e+02,8.110294e-01,4.332410e+00,7.026700e+00,1.901237e-01,4.661335e-01,7.116223e+00
std,2.583418e-01,2.701165e+01,3.713276e+01,2.686434e+01,4.882776e+02,1.366027e+01,1.229784e+01,1.205617e+02,8.184573e-01,3.972706e+00,4.018037e+00,2.560846e-01,5.097858e-01,4.612499e+00
min,0.000000e+00,2.020853e+02,2.794441e+02,4.246918e+02,9.102757e+01,3.014123e+00,4.558969e+00,4.952888e-01,-9.569907e-02,-1.267213e-01,1.728067e-01,-1.294968e-01,-1.326250e-01,2.026558e-05
25%,3.000000e-01,2.543430e+02,3.266345e+02,5.673138e+02,1.744876e+02,8.976903e+00,1.493889e+01,1.076303e+02,3.791428e-01,1.926220e+00,4.530061e+00,5.799794e-02,1.869081e-01,4.429616e+00
50%,5.000000e-01,2.699055e+02,3.456665e+02,5.827677e+02,3.266322e+02,1.742608e+01,2.180278e+01,2.111784e+02,6.681085e-01,3.541990e+00,6.793644e+00,1.420151e-01,3.605243e-01,6.750260e+00
75%,7.000000e-01,2.883478e+02,3.725729e+02,5.933018e+02,7.870456e+02,3.001149e+01,3.317859e+01,3.043738e+02,1.066986e+00,5.741556e+00,9.030967e+00,2.563907e-01,6.126437e-01,9.138975e+00
max,9.000000e-01,4.033092e+02,5.529254e+02,7.004031e+02,3.081163e+03,7.795893e+01,1.298841e+02,5.666409e+02,3.425747e+01,1.253688e+02,1.188544e+02,8.895615e+00,1.651693e+01,1.671498e+02


## Property Prediction
We use MLP multi regressor to predict property of plastic

### Load Model and helpers

In [ ]:

from __future__ import annotations

import ast
import pickle
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split
from tqdm.auto import tqdm

from geoldm import encode, load_model, smiles_to_3d
from geoldm.configs import get_dataset_info
from geoldm.qm9 import dataset

TARGET_COLUMNS = ['Tg', 'Tm', 'Td', 'YM', 'TS_y', 'TS_b', 'eps_b', 'perm_O2', 'perm_CO2', 'perm_He', 'perm_N2', 'perm_CH4', 'perm_H2']


def load_qm9_latent_diffusion(checkpoint_dir: Path):
    """Load a pretrained latent diffusion model and its dataset metadata."""

    args_path = checkpoint_dir / "args.pickle"
    with args_path.open("rb") as handle:
        args = pickle.load(handle)

    # Force CPU execution when CUDA is unavailable.
    if not torch.cuda.is_available():
        setattr(args, "cuda", False)

    dataset_info = get_dataset_info(args.dataset, args.remove_h)
    dataloaders, _ = dataset.retrieve_dataloaders(args)
    train_loader = dataloaders["train"]

    model, nodes_dist, _ = load_model(
        stage="latent_diffusion",
        args=args,
        dataset_info=dataset_info,
        dataloader_train=train_loader,
        checkpoint_path=checkpoint_dir,
    )
    device = next(model.parameters()).device
    return model, dataset_info, nodes_dist, device


def _pick_base_smiles(smiles_entry: object) -> str:
    """Normalize the parquet smiles field to a single SMILES string."""

    if isinstance(smiles_entry, str):
        try:
            maybe_list = ast.literal_eval(smiles_entry)
        except Exception:
            return smiles_entry
        if isinstance(maybe_list, (list, tuple, np.ndarray)):
            for candidate in maybe_list:
                if isinstance(candidate, str):
                    return candidate
    if isinstance(smiles_entry, (list, tuple, np.ndarray)):
        for candidate in smiles_entry:
            if isinstance(candidate, str):
                return candidate
    return str(smiles_entry)


def conformer_to_tensors(
    conformer: Dict,
    dataset_info: Dict,
    device: torch.device,
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor], torch.Tensor, torch.Tensor]:
    """Convert a single RDKit conformer into GeoLDM-ready tensors."""

    atom_decoder: List[str] = dataset_info["atom_decoder"]
    atom_encoder = {symbol: idx for idx, symbol in enumerate(atom_decoder)}
    atom_indices = torch.tensor(
        [atom_encoder[symbol] for symbol in conformer["atom_symbols"]],
        dtype=torch.long,
        device=device,
    )

    one_hot = F.one_hot(atom_indices, num_classes=len(atom_decoder)).float()
    positions = torch.tensor(conformer["coordinates"], dtype=torch.float32, device=device)

    # Add batch dimensions and masks.
    x = positions.unsqueeze(0)
    h = {
        "categorical": one_hot.unsqueeze(0),
        # Use zero charges when absent.
        "integer": torch.zeros(one_hot.shape[0], 1, device=device).unsqueeze(0),
    }

    node_mask = torch.ones(x.shape[0], x.shape[1], 1, device=device)
    edge_mask = node_mask.squeeze(-1)[..., None] * node_mask.squeeze(-1)[:, None]
    return x, h, node_mask, edge_mask


@torch.no_grad()
def smiles_to_latent(
    smiles: str,
    model: torch.nn.Module,
    dataset_info: Dict,
    device: torch.device,
) -> torch.Tensor:
    """Encode a SMILES string into a pooled latent vector using the LDM encoder."""

    conformers = smiles_to_3d(smiles)
    if len(conformers) == 0:
        raise ValueError(f"Failed to build 3D structure for {smiles!r}")

    x, h, node_mask, edge_mask = conformer_to_tensors(conformers[0], dataset_info, device)
    z_x, _, z_h, _ = encode(model, x, h, node_mask=node_mask, edge_mask=edge_mask)

    pooled_x = z_x.mean(dim=1)
    pooled_h = z_h.mean(dim=1)
    latent = torch.cat([pooled_x.flatten(), pooled_h.flatten()], dim=-1)
    return latent.cpu()


def build_latent_matrix(
    frame: pd.DataFrame,
    model: torch.nn.Module,
    dataset_info: Dict,
    device: torch.device,
    *,
    sample_size: int = 512,
    cache_path: Optional[Path] = Path("data/plastic_latents.pt"),
) -> Tuple[torch.Tensor, torch.Tensor, List[int]]:
    """Convert a dataframe of plastics into latent vectors and targets.

    The helper optionally caches results to avoid recomputation when the notebook
    is rerun.
    """

    if cache_path and cache_path.exists():
        payload = torch.load(cache_path)
        return payload["embeddings"], payload["targets"], payload["indices"]

    subset = frame.sample(min(sample_size, len(frame)), random_state=0).reset_index()
    embeddings: List[torch.Tensor] = []
    targets: List[torch.Tensor] = []
    used_indices: List[int] = []

    for _, row in tqdm(subset.iterrows(), total=len(subset), desc="Encoding SMILES"):
        base_smiles = _pick_base_smiles(row["smiles"])
        try:
            latent = smiles_to_latent(base_smiles, model, dataset_info, device)
        except Exception:
            continue

        embeddings.append(latent)
        targets.append(torch.tensor(row[TARGET_COLUMNS].values, dtype=torch.float32))
        used_indices.append(int(row["index"]))

    if not embeddings:
        raise RuntimeError("No embeddings were created; check SMILES parsing")

    emb_tensor = torch.stack(embeddings)
    target_tensor = torch.stack(targets)

    if cache_path:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        torch.save(
            {"embeddings": emb_tensor, "targets": target_tensor, "indices": used_indices},
            cache_path,
        )

    return emb_tensor, target_tensor, used_indices


class MultiTaskMLP(nn.Module):
    def __init__(
        self,
        input_dim: int,
        output_dim: int,
        hidden_sizes: Iterable[int] = (256, 128),
        dropout: float = 0.1,
    ) -> None:
        super().__init__()
        layers: List[nn.Module] = []
        prev = input_dim
        for width in hidden_sizes:
            layers.extend([nn.Linear(prev, width), nn.ReLU(), nn.Dropout(dropout)])
            prev = width
        layers.append(nn.Linear(prev, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:  # noqa: D401
        return self.network(x)


def train_multitask_mlp(
    embeddings: torch.Tensor,
    targets: torch.Tensor,
    *,
    epochs: int = 15,
    batch_size: int = 64,
    lr: float = 1e-3,
    validation_split: float = 0.1,
) -> Tuple[MultiTaskMLP, StandardScaler, StandardScaler, Dict[str, List[float]]]:
    """Train a simple multitask MLP on latent embeddings."""

    x_scaler = StandardScaler()
    y_scaler = StandardScaler()

    x_scaled = x_scaler.fit_transform(embeddings)
    y_scaled = y_scaler.fit_transform(targets)

    features = torch.tensor(x_scaled, dtype=torch.float32)
    labels = torch.tensor(y_scaled, dtype=torch.float32)

    dataset = TensorDataset(features, labels)
    val_size = max(1, int(len(dataset) * validation_split))
    train_size = len(dataset) - val_size
    train_set, val_set = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size)

    model = MultiTaskMLP(features.shape[1], labels.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    history = {"train": [], "val": []}
    for epoch in tqdm(range(epochs), desc="Training epochs"):
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            preds = model(batch_x)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * len(batch_x)
        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                preds = model(batch_x)
                loss = criterion(preds, batch_y)
                val_loss += loss.item() * len(batch_x)
        val_loss /= len(val_loader.dataset)

        history["train"].append(train_loss)
        history["val"].append(val_loss)

    return model, x_scaler, y_scaler, history


def predict_properties(
    smiles: str,
    *,
    ldm_model: torch.nn.Module,
    dataset_info: Dict,
    device: torch.device,
    mlp_model: MultiTaskMLP,
    x_scaler: StandardScaler,
    y_scaler: StandardScaler,
) -> Dict[str, float]:
    """Generate property predictions for a new SMILES string."""

    latent = smiles_to_latent(smiles, ldm_model, dataset_info, device)
    scaled = x_scaler.transform(latent.unsqueeze(0))
    mlp_model.eval()
    with torch.no_grad():
        pred_scaled = mlp_model(torch.tensor(scaled, dtype=torch.float32)).cpu().numpy()
    pred = y_scaler.inverse_transform(pred_scaled)[0]
    return {name: value for name, value in zip(TARGET_COLUMNS, pred)}


In [9]:

# Load pretrained latent diffusion encoder
checkpoint_dir = Path("./qm9_latent2")
ldm_model, dataset_info, nodes_dist, device = load_qm9_latent_diffusion(checkpoint_dir)

# Prepare the plastics dataframe and build (or load) latent embeddings.
plastic_df = pd.read_parquet("data/plastic.parquet")
embeddings, targets, used_indices = build_latent_matrix(
    plastic_df,
    ldm_model,
    dataset_info,
    device,
    sample_size=512,  # keep runtime manageable inside the notebook
    cache_path=Path("data/plastic_latents.pt"),
)

# Train the multi-task MLP regressor.
mlp_model, x_scaler, y_scaler, history = train_multitask_mlp(
    embeddings.numpy(),
    targets.numpy(),
    epochs=10,
    batch_size=64,
    lr=1e-3,
)

print("Training loss trajectory:", history["train"])
print("Validation loss trajectory:", history["val"])


## Optimization 1: Straw for Cold Brew
Optimization Boundary:

Tg: -30\~10 °C

Tm: 120\~200 °C

Td: 250°C 이상

YM: 0.8\~1.8GPa->범위 안에 가두어야 함(1.2 근처)

TS_b: 20MPa 이상-> 최대화 대상(2)

eps_b : 200\~1000%->최대화 대상(1)


In [ ]:

# Example inference call
example_smiles = _pick_base_smiles(plastic_df.loc[0, "smiles"])
predicted = predict_properties(
    example_smiles,
    ldm_model=ldm_model,
    dataset_info=dataset_info,
    device=device,
    mlp_model=mlp_model,
    x_scaler=x_scaler,
    y_scaler=y_scaler,
)

for name, value in predicted.items():
    print(f"{name}: {value:.3f}")
